In [1]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/544.2 kB ? eta -:--:--
   ---------------------------------------- 544.2/544.2 kB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 18.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/30.3 MB ? eta -:--:--
   ----------- ---------------------------- 8.9/30.3 MB 42.6 MB/s eta 0:00:01
   -------------------- ------------------- 15.2/30.3 MB 39.8 MB/s eta 0:00:01
   --------------------------- ------------ 20.7/30.3 MB 32.7 MB/s eta 0:00:01
   ------------------------------ --------- 23.3/30.3 MB 28.9 MB/s eta 0:00:01
   --------------------------------- ------ 25.7/30.3 MB 24.7 MB/s eta 0:00:01
   ----------------------------------- ---- 27.0/30.3 MB 21.4 MB/s eta 0:00:01
   ------------------------------------ --- 28.0/30.3 MB 19.1 MB/s eta 0:00:01
   -----


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import shap
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax

In [3]:

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

bert_model.load_state_dict(torch.load("bert_model.pth", map_location=torch.device('cpu')))

bert_model.eval()

print("BERT model loaded successfully!")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\MIST LAB\AppData\Local\Temp\ipykernel_3124\3988323421.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serial

BERT model loaded successfully!


In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
explainer = shap.Explainer(bert_model, tokenizer)

In [13]:
import pandas as pd
dataset_path = 'preprocessed_english_1.csv'
df = pd.read_csv(dataset_path, encoding='latin-1')
classes = df['label'].unique()
class_to_id = {cls: idx for idx, cls in enumerate(classes)}
id_to_class = {idx: cls for cls, idx in class_to_id.items()}

In [16]:
class_names

['not bully', 'religious', 'others', 'sexual']

In [16]:
def shap_predict(texts):
    # Convert token list back to strings if needed
    cleaned_texts = [
        " ".join(t) if isinstance(t, list) else t
        for t in texts
    ]
    
    # Tokenize and predict
    encodings = tokenizer(cleaned_texts, padding=True, truncation=True, max_length=128, return_tensors="pt")
    
    with torch.no_grad():
        outputs = bert_model(**encodings)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    
    return probs.detach().cpu().numpy()

In [14]:
import shap
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Define masker for text
masker = shap.maskers.Text(tokenizer)

# Create explainer with your custom prediction function
explainer = shap.Explainer(shap_predict, masker)


In [26]:
import shap
masker = shap.maskers.Text(tokenizer)
explainer = shap.Explainer(shap_predict, masker)

text_input = "i hate you, you should die"
shap_values = explainer([text_input])

shap.plots.text(shap_values[0])


In [31]:
text_input = "You're such a pathetic excuse for a human."
shap_values = explainer([text_input])

shap.plots.text(shap_values[0])